In [47]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
veri=pd.read_excel("veri.xlsx")
df=veri.copy()
test=pd.read_excel("id.xlsx")
test1= test.drop(['id'], axis=1)

In [3]:
test1.head()

,customer,item,date
0,A,0,2020-10
1,A,0,2020-11
2,A,0,2020-12
3,A,0,2020-8
4,A,0,2020-9


In [4]:
df.head()

,customer,item,date,order
0,B,39,2018-1,369.20
1,B,57,2018-11,1252.80
2,B,11,2020-6,1036.00
3,A,53,2018-12,51.84
4,A,1,2019-2,72.00


In [5]:
df.shape

(12583, 4)

In [6]:
df_new=pd.DataFrame(columns = ['customer', 'item', 'year_month', 'month', 'year',  'order'])

In [7]:
grup=df.groupby(["customer","item","date"])

In [8]:
for i in grup:
    a,i=i
    i["order"]=i["order"].sum()
    df_new=df_new.append(i,ignore_index=False)
    

In [9]:
df_new=df_new.drop_duplicates()
df_new=df_new.reset_index(drop=True)

In [10]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = pd.DatetimeIndex(test['date']).year
test['month'] = pd.DatetimeIndex(test['date']).month
test=test.drop(["date"],axis=1)

In [11]:
test1['date'] = pd.to_datetime(test1['date'])
test1['year'] = pd.DatetimeIndex(test1['date']).year
test1['month'] = pd.DatetimeIndex(test1['date']).month
test1=test1.drop(["date"],axis=1)

In [12]:
df_new['date'] = pd.to_datetime(df_new['date'])
df_new['year'] = pd.DatetimeIndex(df_new['date']).year
df_new['month'] = pd.DatetimeIndex(df_new['date']).month
df_new=df_new.drop(["date"],axis=1)

In [13]:
df_new.shape

(2382, 6)

In [14]:
df_new.head()

,customer,item,year_month,month,year,order
0,A,0,NaN,1,2018,216.0
1,A,0,NaN,10,2018,576.0
2,A,0,NaN,11,2018,144.0
3,A,0,NaN,12,2018,72.0
4,A,0,NaN,2,2018,576.0


In [15]:
new_encoding=[]
for i in range (0, len(test)):
    if test["year"][i]==2020:
        x=24+test["month"][i]
        new_encoding.append(x)
    if test["year"][i]==2021:
        x=36+test["month"][i]
        new_encoding.append(x)
new_encoding=pd.DataFrame(new_encoding,columns=["year_month"])
test["year_month"]=new_encoding["year_month"]

In [16]:
new_encoding=[]
for i in range (0, len(test1)):
    if test1["year"][i]==2020:
        x=24+test1["month"][i]
        new_encoding.append(x)
    if test1["year"][i]==2021:
        x=36+test1["month"][i]
        new_encoding.append(x)
new_encoding=pd.DataFrame(new_encoding,columns=["year_month"])
test1["year_month"]=new_encoding["year_month"]   

In [17]:
new_encoding1 = []
for i in range(0, len(df_new)): 
    if df_new['year'][i] == 2018:
        x = 0 + df_new['month'][i]
        new_encoding1.append(x)
    if df_new['year'][i] == 2019:
        x = 12 + df_new['month'][i]
        new_encoding1.append(x)
    if df_new['year'][i] == 2020:
        x = 24 + df_new['month'][i]
        new_encoding1.append(x)
        
new_encoding1=pd.DataFrame(new_encoding1,columns=["year_month"])

In [18]:
df_new["year_month"]=new_encoding1["year_month"]

In [19]:
df_new=pd.get_dummies(df_new,columns=["customer"],prefix=["customer"])

In [20]:
test1=pd.get_dummies(test1,columns=["customer"],prefix=["customer"])

In [21]:
df_new=pd.get_dummies(df_new,columns=["item"],prefix=["item"])

In [22]:
test1=pd.get_dummies(test1,columns=["item"],prefix=["item"])

In [23]:
new_encoding=[]
for i in range(0,len(df_new)):
    if df_new["year"][i]==2018:
        new_encoding.append(0)
    elif df_new["year"][i]==2019:
        new_encoding.append(1)
    elif df_new["year"][i]==2020:
        new_encoding.append(2)
new_encoding=pd.DataFrame(new_encoding,columns=["year"])
df_new["year"]=new_encoding["year"]

In [24]:
new_encoding=[]
for i in range(0,len(test1)):
    if test1["year"][i]==2020:
        new_encoding.append(2)
    elif test1["year"][i]==2021:
        new_encoding.append(3)
new_encoding=pd.DataFrame(new_encoding,columns=["year"])
test1["year"]=new_encoding["year"]

In [25]:
df_new.head()

,year_month,month,year,order,customer_A,customer_B,customer_C,item_0,item_1,item_2,...,item_73,item_74,item_75,item_76,item_77,item_78,item_79,item_80,item_81,item_82
0,1,1,0,216.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,10,0,576.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,11,0,144.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12,12,0,72.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2,0,576.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
test1.head()

,year,month,year_month,customer_A,customer_B,customer_C,item_0,item_1,item_2,item_3,...,item_73,item_74,item_75,item_76,item_77,item_78,item_79,item_80,item_81,item_82
0,2,10,34,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,11,35,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,12,36,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,8,32,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,9,33,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   item        498 non-null    int64
 1   year        498 non-null    int64
 2   month       498 non-null    int64
 3   year_month  498 non-null    int64
 4   customer_A  498 non-null    uint8
 5   customer_B  498 non-null    uint8
 6   customer_C  498 non-null    uint8
dtypes: int64(4), uint8(3)
memory usage: 17.1 KB


In [40]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2382 entries, 0 to 2381
Data columns (total 90 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year_month  2382 non-null   int64  
 1   month       2382 non-null   int64  
 2   year        2382 non-null   int64  
 3   order       2382 non-null   float64
 4   customer_A  2382 non-null   uint8  
 5   customer_B  2382 non-null   uint8  
 6   customer_C  2382 non-null   uint8  
 7   item_0      2382 non-null   uint8  
 8   item_1      2382 non-null   uint8  
 9   item_2      2382 non-null   uint8  
 10  item_3      2382 non-null   uint8  
 11  item_4      2382 non-null   uint8  
 12  item_5      2382 non-null   uint8  
 13  item_6      2382 non-null   uint8  
 14  item_7      2382 non-null   uint8  
 15  item_8      2382 non-null   uint8  
 16  item_9      2382 non-null   uint8  
 17  item_10     2382 non-null   uint8  
 18  item_11     2382 non-null   uint8  
 19  item_12     2382 non-null  

In [92]:
def RMSE(df,order,mdl):
    X=df.drop(["order"],axis=1)
    y=df[[order]]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
    #modelleme
    
    model=mdl().fit(X_train,y_train)
    predict_y=model.predict(X_test)
    RMSE_deger=np.sqrt(mean_squared_error(y_test,predict_y))
    model_ismi=mdl.__name__
    print(model_ismi,":","RMSE=",RMSE_deger)
    

In [94]:
models=[GradientBoostingRegressor,
       RandomForestRegressor,
       DecisionTreeRegressor,
       KNeighborsRegressor,
       SVR,
       Ridge,
       LinearRegression,]

In [95]:
for i in models:
    RMSE(df_new,"order",i)

GradientBoostingRegressor : RMSE= 1499.017279367573
RandomForestRegressor : RMSE= 1618.6200703699944
DecisionTreeRegressor : RMSE= 1957.9511132546954
KNeighborsRegressor : RMSE= 2305.022454003029
SVR : RMSE= 2313.4030728134517
Ridge : RMSE= 1539.4223466381302
LinearRegression : RMSE= 1539.2462251044408


In [56]:
X=df_new.drop(["order"],axis=1)
y=df_new[["order"]]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [57]:
GB_model=GradientBoostingRegressor().fit(X,y)

In [62]:
?GB_model

Type:        GradientBoostingRegressor
String form: GradientBoostingRegressor()
Length:      100
File:        c:\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py
Docstring:  
Gradient Boosting for regression.

GB builds an additive model in a forward stage-wise fashion;
it allows for the optimization of arbitrary differentiable loss functions.
In each stage a regression tree is fit on the negative gradient of the
given loss function.

Read more in the :ref:`User Guide <gradient_boosting>`.

Parameters
----------
loss : {'ls', 'lad', 'huber', 'quantile'}, default='ls'
    loss function to be optimized. 'ls' refers to least squares
    regression. 'lad' (least absolute deviation) is a highly robust
    loss function solely based on order information of the input
    variables. 'huber' is a combination of the two. 'quantile'
    allows quantile regression (use `alpha` to specify the quantile).

learning_rate : float, default=0.1
    learning rate shrinks the contribution of each tree b

In [79]:
GB_params={"learning_rate":[0.01,0.1,0.5,0.8],
          "max_depth":[3,5,8,10],
          "n_estimators":[100,200,400,600],
          "subsample":[1,0.5,0.8],
          "loss":["ls","lad","quantile"]}

In [80]:
GB_cv_models=GridSearchCV(GB_model,
                          GB_params,
                          cv=5,
                          verbose=2,n_jobs=-1).fit(X_train,y_train)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 18.7min finished


In [81]:
GB_cv_models.best_params_

{'learning_rate': 0.1,
 'loss': 'lad',
 'max_depth': 8,
 'n_estimators': 200,
 'subsample': 0.5}

In [82]:
GB_model=GradientBoostingRegressor(learning_rate=0.1,loss="lad",max_depth=8,n_estimators=200,subsample=0.5).fit(X,y)

In [89]:
GB_ilkel_model=GradientBoostingRegressor().fit(X,y)

In [ ]:
#İLKEL TEST HATASI

In [90]:
y_pred_ilkel=GB_ilkel_model.predict(X_test)

In [91]:
RMSE_deger=np.sqrt(mean_squared_error(y_test,y_pred_ilkel))
RMSE_deger 

1295.4707676201122

In [85]:
#MODEL TUNING TEST HATASI

In [83]:
y_pred=GB_model.predict(X_test) #test için ayrılan verinin tahmini

In [86]:
RMSE_deger=np.sqrt(mean_squared_error(y_test,y_pred))
RMSE_deger 

1121.030608198787

In [84]:
predict_y=GB_model.predict(test1) #istenilen 6 aylık veri için tahmin

In [87]:
predict_y=pd.DataFrame(predict_y,columns=["order"])
test["order"]=predict_y["order"]

In [88]:
test.to_csv('6_aylik_tahmin.csv', columns=['id', 'order'], index=False)